In [ ]:
# Import required libraries
import sys
sys.path.append('../src')

import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import time
from pathlib import Path

from src.dataset import prepare_dataset, get_dataloader
from src.evaluate import ModelEvaluator, load_model, compare_models
from src.model.gpt2_custom import MaskedGPT2LMHeadModel
from src.model.schedule import load_mask_schedule, create_predefined_schedules, visualize_schedule
from src.model.mask_utils import (
    create_layer_wise_masks, 
    get_attention_pattern_efficiency, 
    visualize_attention_pattern,
    compare_attention_patterns
)

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Set style
plt.style.use('seaborn-v0_8')
sns.set_palette("Set2")
plt.rcParams['figure.figsize'] = (12, 8)

## 1. Load Masking Schedules

In [ ]:
# Load and display available masking schedules
schedule_config_path = '../config/schedule_config.json'

try:
    with open(schedule_config_path, 'r') as f:
        schedule_config = json.load(f)
    print("✓ Loaded schedule configuration")
except FileNotFoundError:
    print("⚠ Schedule config not found, creating predefined schedules...")
    from src.model.schedule import save_predefined_schedules
    save_predefined_schedules(schedule_config_path)
    
    with open(schedule_config_path, 'r') as f:
        schedule_config = json.load(f)
    print("✓ Created and loaded predefined schedules")

print(f"\nAvailable schedules: {list(schedule_config['schedules'].keys())}")

# Display schedule details
for name, schedule in schedule_config['schedules'].items():
    print(f"\n{name.upper()}: {schedule['description']}")
    
    # Count masked layers
    masked_layers = sum(1 for layer_config in schedule['layers'].values() 
                       if layer_config['mask_type'] != 'none')
    total_layers = len(schedule['layers'])
    
    print(f"  Masked layers: {masked_layers}/{total_layers}")
    
    # Show layer-wise configuration
    print(f"  Layer configuration:")
    for layer_idx, layer_config in list(schedule['layers'].items())[:6]:  # Show first 6
        mask_type = layer_config['mask_type']
        window_size = layer_config.get('window_size', 'N/A')
        print(f"    Layer {layer_idx}: {mask_type} (window: {window_size})")
    
    if total_layers > 6:
        print(f"    ... (showing 6/{total_layers} layers)")

## 2. Visualize Masking Schedules

In [ ]:
# Create and visualize masks for different schedules
seq_length = 256  # Use smaller size for visualization
num_layers = 12

fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.flatten()

schedule_names = ['full', 'half', 'quarter', 'aggressive']

for idx, schedule_name in enumerate(schedule_names):
    if idx >= len(axes):
        break
    
    schedule_data = schedule_config['schedules'][schedule_name]
    
    # Create masks for all layers
    masks = create_layer_wise_masks(seq_length, num_layers, schedule_data['layers'])
    
    # Calculate efficiency statistics
    total_attended = 0
    total_possible = 0
    
    for layer_idx, mask in masks.items():
        efficiency = get_attention_pattern_efficiency(mask)
        total_attended += efficiency['attended_positions']
        total_possible += efficiency['total_positions']
    
    overall_sparsity = 1 - (total_attended / total_possible)
    memory_reduction = total_possible / total_attended
    
    # Visualize last layer mask as representative
    last_layer_mask = masks[num_layers - 1].cpu().numpy()
    
    im = axes[idx].imshow(last_layer_mask, cmap='Blues', aspect='equal')
    axes[idx].set_title(f'{schedule_name.upper()}\n'
                       f'Sparsity: {overall_sparsity:.2f}, '
                       f'Memory: {memory_reduction:.1f}x')
    axes[idx].set_xlabel('Key Position')
    axes[idx].set_ylabel('Query Position')
    
    # Add colorbar
    plt.colorbar(im, ax=axes[idx], fraction=0.046)

plt.suptitle('Attention Masking Patterns (Last Layer)', fontsize=16)
plt.tight_layout()
plt.savefig('../results/figures/masking_schedules_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Create layer-wise progression visualization
schedule_name = 'half'  # Focus on half schedule for detailed analysis
schedule_data = schedule_config['schedules'][schedule_name]

# Create masks for all layers
masks = create_layer_wise_masks(128, num_layers, schedule_data['layers'])

# Calculate sparsity progression across layers
layer_sparsity = []
layer_window_sizes = []

for layer_idx in range(num_layers):
    mask = masks[layer_idx]
    efficiency = get_attention_pattern_efficiency(mask)
    layer_sparsity.append(efficiency['sparsity'])
    
    # Get window size for this layer
    layer_config = schedule_data['layers'][str(layer_idx)]
    window_size = layer_config.get('window_size')
    layer_window_sizes.append(window_size if window_size else 128)  # Use seq_length as default

# Plot layer-wise statistics
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Sparsity progression
axes[0].plot(range(num_layers), layer_sparsity, 'o-', linewidth=2, markersize=6)
axes[0].set_title(f'Sparsity Progression: {schedule_name.upper()}')
axes[0].set_xlabel('Layer Index')
axes[0].set_ylabel('Attention Sparsity')
axes[0].grid(True, alpha=0.3)
axes[0].set_ylim(-0.05, 1.05)

# Window size progression
axes[1].semilogy(range(num_layers), layer_window_sizes, 'o-', linewidth=2, markersize=6, color='orange')
axes[1].set_title(f'Window Size Progression: {schedule_name.upper()}')
axes[1].set_xlabel('Layer Index')
axes[1].set_ylabel('Window Size (log scale)')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../results/figures/layer_wise_progression.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"Layer-wise sparsity for {schedule_name}:")
for i, sparsity in enumerate(layer_sparsity):
    window_size = layer_window_sizes[i]
    print(f"  Layer {i:2d}: {sparsity:.3f} sparsity (window: {window_size})")

## 3. Load Masked Models

In [ ]:
# Attempt to load trained masked models
masked_models = {}
schedule_names = ['half', 'quarter', 'aggressive']  # Skip 'full' as it's equivalent to baseline

for schedule_name in schedule_names:
    model_path = f'../models/mask_gpt2/{schedule_name}'
    
    try:
        print(f"Loading {schedule_name} model from {model_path}...")
        model, tokenizer = load_model(model_path, 'masked')
        masked_models[schedule_name] = {
            'model': model,
            'tokenizer': tokenizer,
            'path': model_path,
            'schedule': schedule_config['schedules'][schedule_name]
        }
        print(f"✓ Loaded {schedule_name} model")
    except Exception as e:
        print(f"⚠ Could not load {schedule_name} model: {e}")
        print(f"Creating model with {schedule_name} schedule for demonstration...")
        
        # Create model with the schedule for demonstration
        try:
            from transformers import GPT2Config
            config = GPT2Config.from_pretrained('gpt2')
            schedule_data = schedule_config['schedules'][schedule_name]
            
            model = MaskedGPT2LMHeadModel.from_pretrained('gpt2', mask_schedule=schedule_data)
            tokenizer = model.transformer.wte.weight.data.new_zeros(1).device
            from transformers import GPT2Tokenizer
            tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
            tokenizer.pad_token = tokenizer.eos_token
            
            masked_models[schedule_name] = {
                'model': model,
                'tokenizer': tokenizer,
                'path': 'gpt2',  # Base model
                'schedule': schedule_data
            }
            print(f"✓ Created {schedule_name} model for demonstration")
        except Exception as e2:
            print(f"✗ Failed to create {schedule_name} model: {e2}")

print(f"\nLoaded {len(masked_models)} masked models")
for name in masked_models.keys():
    print(f"  - {name}")

## 4. Model Information and Schedule Analysis

In [ ]:
# Display model information and mask schedule summaries
for name, model_info in masked_models.items():
    model = model_info['model']
    schedule = model_info['schedule']
    
    print(f"\n{'='*60}")
    print(f"MODEL: {name.upper()}")
    print(f"{'='*60}")
    
    # Basic model info
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    
    print(f"Model path: {model_info['path']}")
    print(f"Total parameters: {total_params:,}")
    print(f"Trainable parameters: {trainable_params:,}")
    print(f"Model size: {total_params * 4 / 1024**2:.1f} MB")
    
    # Schedule summary
    print(f"\nMask Schedule: {schedule['description']}")
    
    # Calculate schedule efficiency
    try:
        masks = create_layer_wise_masks(512, 12, schedule['layers'])
        
        total_attended = 0
        total_possible = 0
        masked_layers = 0
        
        for layer_idx, mask in masks.items():
            efficiency = get_attention_pattern_efficiency(mask)
            total_attended += efficiency['attended_positions']
            total_possible += efficiency['total_positions']
            
            # Check if layer is masked
            layer_config = schedule['layers'][str(layer_idx)]
            if layer_config['mask_type'] != 'none':
                masked_layers += 1
        
        overall_sparsity = 1 - (total_attended / total_possible)
        memory_reduction = total_possible / total_attended
        
        print(f"Masked layers: {masked_layers}/12")
        print(f"Overall sparsity: {overall_sparsity:.3f}")
        print(f"Memory reduction: {memory_reduction:.2f}x")
        
        # Show some layer details
        print(f"\nLayer details (first 6):")
        for layer_idx in range(min(6, len(masks))):
            layer_config = schedule['layers'][str(layer_idx)]
            mask = masks[layer_idx]
            efficiency = get_attention_pattern_efficiency(mask)
            
            mask_type = layer_config['mask_type']
            window_size = layer_config.get('window_size', 'N/A')
            sparsity = efficiency['sparsity']
            
            print(f"  Layer {layer_idx}: {mask_type:15s} (window: {str(window_size):>4s}, sparsity: {sparsity:.3f})")
            
    except Exception as e:
        print(f"Could not calculate schedule efficiency: {e}")

## 5. Prepare Test Datasets

In [ ]:
# Prepare test datasets for evaluation
print("Preparing test datasets...")

# Use smaller datasets for faster evaluation in this notebook
tiny_datasets, _ = prepare_dataset('tiny_stories', max_length=512)
tiny_test_loader = get_dataloader(
    tiny_datasets['test'], 
    batch_size=4,  # Smaller batch for faster processing
    shuffle=False
)

print(f"✓ Prepared TinyStories test dataset")
print(f"  Test examples: {len(tiny_datasets['test'])}")
print(f"  Batch size: 4")
print(f"  Max sequence length: 512")

# Optionally prepare WikiText for more comprehensive analysis
try:
    wiki_datasets, _ = prepare_dataset('wikitext-103', max_length=1024)
    wiki_test_loader = get_dataloader(
        wiki_datasets['test'][:100],  # Use subset for faster evaluation
        batch_size=2,
        shuffle=False
    )
    print(f"✓ Prepared WikiText-103 test subset (100 examples)")
    test_datasets = {
        'tiny_stories': tiny_test_loader,
        'wikitext': wiki_test_loader
    }
except Exception as e:
    print(f"⚠ Could not prepare WikiText dataset: {e}")
    print("Continuing with TinyStories only...")
    test_datasets = {
        'tiny_stories': tiny_test_loader
    }

## 6. Model Evaluation

In [ ]:
# Evaluate masked models
evaluation_results = {}

for model_name, model_info in masked_models.items():
    model = model_info['model']
    tokenizer = model_info['tokenizer']
    
    print(f"\n{'='*50}")
    print(f"EVALUATING: {model_name.upper()}")
    print(f"{'='*50}")
    
    evaluator = ModelEvaluator(model, tokenizer, device)
    evaluation_results[model_name] = {}
    
    # Evaluate on available datasets
    for dataset_name, test_loader in test_datasets.items():
        print(f"\n--- Evaluating on {dataset_name} ---")
        
        try:
            # Quick evaluation (fewer batches for notebook)
            print("Calculating perplexity...")
            # Use subset of data for faster evaluation
            perplexity = evaluator.calculate_perplexity(
                iter(list(test_loader)[:20])  # Use first 20 batches
            )
            
            print("Measuring throughput...")
            throughput = evaluator.measure_throughput(
                test_loader, num_batches=10  # Use fewer batches
            )
            
            print("Measuring latency...")
            latency = evaluator.measure_latency(
                input_length=256, num_trials=20  # Fewer trials
            )
            
            print("Measuring memory usage...")
            memory = evaluator.measure_memory_usage()
            
            # Store results
            evaluation_results[model_name][dataset_name] = {
                'perplexity': perplexity,
                'throughput': throughput,
                'latency': latency,
                'memory': memory
            }
            
            # Print summary
            print(f"Results:")
            print(f"  Perplexity: {perplexity:.2f}")
            print(f"  Throughput: {throughput['tokens_per_second']:.1f} tokens/sec")
            print(f"  Latency: {latency['mean_latency_ms']:.1f} ms")
            if 'gpu_memory_allocated_mb' in memory:
                print(f"  GPU Memory: {memory['gpu_memory_allocated_mb']:.1f} MB")
            
        except Exception as e:
            print(f"Error evaluating {model_name} on {dataset_name}: {e}")
            evaluation_results[model_name][dataset_name] = None

print("\n✓ Masked model evaluation completed")

## 7. Load Baseline Results for Comparison

In [ ]:
# Try to load baseline results for comparison
baseline_results = {}

try:
    with open('../results/baseline/comprehensive_evaluation.json', 'r') as f:
        baseline_data = json.load(f)
        baseline_results = baseline_data.get('evaluation_results', {})
    print("✓ Loaded baseline evaluation results")
except FileNotFoundError:
    print("⚠ No baseline results found, will create comparison with current data only")
    
    # Create placeholder baseline results using GPT-2
    try:
        from transformers import GPT2LMHeadModel, GPT2Tokenizer
        
        baseline_model = GPT2LMHeadModel.from_pretrained('gpt2')
        baseline_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
        baseline_tokenizer.pad_token = baseline_tokenizer.eos_token
        
        print("Creating baseline comparison with pretrained GPT-2...")
        baseline_evaluator = ModelEvaluator(baseline_model, baseline_tokenizer, device)
        
        baseline_results['baseline_gpt2'] = {}
        
        for dataset_name, test_loader in test_datasets.items():
            print(f"Evaluating baseline on {dataset_name}...")
            
            # Quick baseline evaluation
            perplexity = baseline_evaluator.calculate_perplexity(
                iter(list(test_loader)[:20])
            )
            throughput = baseline_evaluator.measure_throughput(
                test_loader, num_batches=10
            )
            latency = baseline_evaluator.measure_latency(
                input_length=256, num_trials=20
            )
            memory = baseline_evaluator.measure_memory_usage()
            
            baseline_results['baseline_gpt2'][dataset_name] = {
                'perplexity': perplexity,
                'throughput': throughput,
                'latency': latency,
                'memory': memory
            }
            
            print(f"  Baseline perplexity: {perplexity:.2f}")
        
        print("✓ Created baseline comparison results")
        
    except Exception as e:
        print(f"Could not create baseline comparison: {e}")

# Combine all results
all_results = {**baseline_results, **evaluation_results}
print(f"\nTotal models for comparison: {len(all_results)}")
for model_name in all_results.keys():
    print(f"  - {model_name}")

## 8. Performance Comparison Analysis

In [ ]:
# Create comprehensive comparison
def extract_comparison_metrics(results):
    comparison_data = []
    
    for model_name, model_results in results.items():
        for dataset_name, dataset_results in model_results.items():
            if dataset_results is not None:
                row = {
                    'model': model_name,
                    'dataset': dataset_name,
                    'perplexity': dataset_results['perplexity'],
                    'throughput': dataset_results['throughput']['tokens_per_second'],
                    'latency': dataset_results['latency']['mean_latency_ms'],
                    'memory': dataset_results['memory'].get('gpu_memory_allocated_mb', 0),
                    'model_type': 'baseline' if 'baseline' in model_name else 'masked'
                }
                comparison_data.append(row)
    
    return pd.DataFrame(comparison_data)

comparison_df = extract_comparison_metrics(all_results)
print("Comparison Results:")
print(comparison_df)

# Calculate relative performance vs baseline
if not comparison_df.empty and 'baseline_gpt2' in comparison_df['model'].values:
    print("\n=== Relative Performance vs Baseline ===")
    
    for dataset in comparison_df['dataset'].unique():
        dataset_df = comparison_df[comparison_df['dataset'] == dataset]
        baseline_row = dataset_df[dataset_df['model'].str.contains('baseline')]
        
        if not baseline_row.empty:
            baseline_ppl = baseline_row['perplexity'].iloc[0]
            baseline_throughput = baseline_row['throughput'].iloc[0]
            baseline_latency = baseline_row['latency'].iloc[0]
            
            print(f"\n{dataset.upper()}:")
            print(f"{'Model':<15} {'PPL Ratio':<12} {'Speed Ratio':<12} {'Latency Ratio':<15}")
            print("-" * 60)
            
            for _, row in dataset_df.iterrows():
                if 'baseline' not in row['model']:
                    ppl_ratio = row['perplexity'] / baseline_ppl
                    speed_ratio = row['throughput'] / baseline_throughput
                    latency_ratio = row['latency'] / baseline_latency
                    
                    print(f"{row['model']:<15} {ppl_ratio:<12.3f} {speed_ratio:<12.3f} {latency_ratio:<15.3f}")
                    
                    # Interpretation
                    if ppl_ratio < 1.1 and speed_ratio > 1.1:
                        print(f"  → ✓ Good trade-off: minimal quality loss, speed gain")
                    elif ppl_ratio > 1.5:
                        print(f"  → ⚠ Significant quality degradation")
                    elif speed_ratio < 1.1:
                        print(f"  → ⚠ No significant speed improvement")
                    else:
                        print(f"  → 🤔 Mixed results")

## 9. Visualization of Results

In [ ]:
# Create comprehensive visualization
if not comparison_df.empty:
    # Set up the plotting
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    axes = axes.flatten()
    
    # Color palette for model types
    palette = {'baseline': '#1f77b4', 'masked': '#ff7f0e'}
    
    # 1. Perplexity comparison
    sns.barplot(data=comparison_df, x='dataset', y='perplexity', 
                hue='model_type', ax=axes[0], palette=palette)
    axes[0].set_title('Perplexity Comparison')
    axes[0].set_ylabel('Perplexity (lower is better)')
    
    # 2. Throughput comparison
    sns.barplot(data=comparison_df, x='dataset', y='throughput', 
                hue='model_type', ax=axes[1], palette=palette)
    axes[1].set_title('Throughput Comparison')
    axes[1].set_ylabel('Tokens/Second (higher is better)')
    
    # 3. Latency comparison
    sns.barplot(data=comparison_df, x='dataset', y='latency', 
                hue='model_type', ax=axes[2], palette=palette)
    axes[2].set_title('Latency Comparison')
    axes[2].set_ylabel('Latency ms (lower is better)')
    
    # 4. Memory usage
    if comparison_df['memory'].sum() > 0:
        sns.barplot(data=comparison_df, x='dataset', y='memory', 
                    hue='model_type', ax=axes[3], palette=palette)
        axes[3].set_title('Memory Usage')
        axes[3].set_ylabel('GPU Memory MB')
    else:
        axes[3].text(0.5, 0.5, 'GPU Memory\nData Not Available', 
                    transform=axes[3].transAxes, ha='center', va='center')
        axes[3].set_title('Memory Usage')
    
    # 5. Efficiency vs Quality scatter plot
    if len(comparison_df) > 1:
        # Calculate efficiency score (inverse of latency)
        comparison_df['efficiency'] = 1000 / comparison_df['latency']  
        
        for dataset in comparison_df['dataset'].unique():
            dataset_data = comparison_df[comparison_df['dataset'] == dataset]
            
            for model_type in dataset_data['model_type'].unique():
                type_data = dataset_data[dataset_data['model_type'] == model_type]
                axes[4].scatter(type_data['efficiency'], 1/type_data['perplexity'], 
                              label=f'{dataset}_{model_type}', alpha=0.7, s=100)
        
        axes[4].set_xlabel('Efficiency (1000/latency)')
        axes[4].set_ylabel('Quality (1/perplexity)')
        axes[4].set_title('Efficiency vs Quality Trade-off')
        axes[4].legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    
    # 6. Model comparison by schedule
    masked_only = comparison_df[comparison_df['model_type'] == 'masked']
    if not masked_only.empty:
        sns.barplot(data=masked_only, x='model', y='perplexity', ax=axes[5])
        axes[5].set_title('Masked Models Perplexity')
        axes[5].set_ylabel('Perplexity')
        axes[5].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.savefig('../results/figures/masked_models_comparison.png', dpi=300, bbox_inches='tight')
    plt.show()
else:
    print("No comparison data available for visualization")

## 10. Attention Pattern Analysis

In [ ]:
# Analyze attention patterns for one masked model
if masked_models:
    # Use the first available masked model
    model_name = list(masked_models.keys())[0]
    model_info = masked_models[model_name]
    model = model_info['model'].to(device)
    tokenizer = model_info['tokenizer']
    schedule = model_info['schedule']
    
    print(f"Analyzing attention patterns for: {model_name}")
    
    # Sample text for analysis
    sample_text = "Once upon a time, there was a brave little mouse who lived in a big house."
    
    # Tokenize
    inputs = tokenizer(sample_text, return_tensors='pt', truncation=True, max_length=128)
    input_ids = inputs['input_ids'].to(device)
    tokens = input_ids[0].cpu().numpy()
    
    print(f"Sample text: '{sample_text}'")
    print(f"Tokens ({len(tokens)}): {[tokenizer.decode([token]) for token in tokens]}")
    
    # Extract attention from model
    model.eval()
    with torch.no_grad():
        try:
            outputs = model(input_ids, output_attentions=True)
            all_attentions = outputs.attentions  # List of attention tensors
            
            print(f"\nExtracted attention from {len(all_attentions)} layers")
            
            # Visualize attention patterns for different layers
            layers_to_show = [0, 5, 11]  # First, middle, last
            fig, axes = plt.subplots(1, len(layers_to_show), figsize=(15, 5))
            
            if len(layers_to_show) == 1:
                axes = [axes]
            
            for i, layer_idx in enumerate(layers_to_show):
                if layer_idx < len(all_attentions):
                    attention = all_attentions[layer_idx][0, 0].cpu().numpy()  # First head
                    
                    im = axes[i].imshow(attention, cmap='Blues', aspect='equal')
                    axes[i].set_title(f'Layer {layer_idx}\n{schedule["layers"][str(layer_idx)]["mask_type"]}')
                    axes[i].set_xlabel('Key Position')
                    axes[i].set_ylabel('Query Position')
                    
                    plt.colorbar(im, ax=axes[i], fraction=0.046)
            
            plt.suptitle(f'Attention Patterns: {model_name.upper()}')
            plt.tight_layout()
            plt.savefig(f'../results/figures/{model_name}_attention_patterns.png', 
                       dpi=300, bbox_inches='tight')
            plt.show()
            
            # Calculate attention statistics
            print("\nAttention Statistics by Layer:")
            print(f"{'Layer':<6} {'Type':<15} {'Window':<8} {'Sparsity':<10} {'Entropy':<10}")
            print("-" * 55)
            
            for layer_idx, attention_tensor in enumerate(all_attentions[:6]):  # Show first 6
                attention = attention_tensor[0, 0].cpu().numpy()  # First head
                
                # Calculate metrics
                sparsity = np.mean(attention < 0.01)
                entropy = -np.sum(attention * np.log(attention + 1e-10), axis=-1).mean()
                
                # Get layer config
                layer_config = schedule['layers'][str(layer_idx)]
                mask_type = layer_config['mask_type']
                window_size = layer_config.get('window_size', 'N/A')
                
                print(f"{layer_idx:<6} {mask_type:<15} {str(window_size):<8} {sparsity:<10.3f} {entropy:<10.3f}")
                
        except Exception as e:
            print(f"Could not extract attention patterns: {e}")
            print("This might be because the model doesn't support attention output")
else:
    print("No masked models available for attention analysis")

## 11. Summary and Conclusions

In [ ]:
# Create final summary
print("\n" + "="*80)
print("MASK SCHEDULE EVALUATION SUMMARY")
print("="*80)

# Schedule efficiency summary
print("\n1. MASKING SCHEDULE EFFICIENCY:")
schedule_efficiency = {}

for schedule_name in ['full', 'half', 'quarter', 'aggressive']:
    if schedule_name in schedule_config['schedules']:
        schedule_data = schedule_config['schedules'][schedule_name]
        
        try:
            masks = create_layer_wise_masks(512, 12, schedule_data['layers'])
            
            total_attended = sum(
                get_attention_pattern_efficiency(mask)['attended_positions']
                for mask in masks.values()
            )
            total_possible = sum(
                get_attention_pattern_efficiency(mask)['total_positions']
                for mask in masks.values()
            )
            
            overall_sparsity = 1 - (total_attended / total_possible)
            memory_reduction = total_possible / total_attended
            
            schedule_efficiency[schedule_name] = {
                'sparsity': overall_sparsity,
                'memory_reduction': memory_reduction
            }
            
            print(f"  {schedule_name.upper():<12}: {overall_sparsity:.3f} sparsity, {memory_reduction:.2f}x memory reduction")
        
        except Exception as e:
            print(f"  {schedule_name.upper():<12}: Could not calculate ({e})")

# Performance summary
if not comparison_df.empty:
    print("\n2. PERFORMANCE COMPARISON:")
    
    # Find best performing masked model
    masked_models_df = comparison_df[comparison_df['model_type'] == 'masked']
    baseline_models_df = comparison_df[comparison_df['model_type'] == 'baseline']
    
    if not masked_models_df.empty and not baseline_models_df.empty:
        for dataset in comparison_df['dataset'].unique():
            print(f"\n  {dataset.upper()}:")
            
            dataset_masked = masked_models_df[masked_models_df['dataset'] == dataset]
            dataset_baseline = baseline_models_df[baseline_models_df['dataset'] == dataset]
            
            if not dataset_baseline.empty:
                baseline_ppl = dataset_baseline['perplexity'].iloc[0]
                baseline_speed = dataset_baseline['throughput'].iloc[0]
                
                print(f"    Baseline perplexity: {baseline_ppl:.2f}")
                print(f"    Baseline speed: {baseline_speed:.1f} tokens/sec")
                
                if not dataset_masked.empty:
                    best_quality = dataset_masked.loc[dataset_masked['perplexity'].idxmin()]
                    best_speed = dataset_masked.loc[dataset_masked['throughput'].idxmax()]
                    
                    print(f"    Best quality: {best_quality['model']} (PPL: {best_quality['perplexity']:.2f})")
                    print(f"    Best speed: {best_speed['model']} ({best_speed['throughput']:.1f} tokens/sec)")
                    
                    # Calculate improvements
                    quality_degradation = (best_quality['perplexity'] / baseline_ppl - 1) * 100
                    speed_improvement = (best_speed['throughput'] / baseline_speed - 1) * 100
                    
                    print(f"    Quality change: {quality_degradation:+.1f}%")
                    print(f"    Speed improvement: {speed_improvement:+.1f}%")

# Recommendations
print("\n3. RECOMMENDATIONS:")

recommendations = [
    "📊 Data Collection: Run full training and evaluation on larger datasets",
    "🔬 Hyperparameter Tuning: Fine-tune learning rates and schedules for masked models",
    "📏 Sequence Length Analysis: Test performance on different sequence lengths",
    "🎯 Task-Specific Evaluation: Test on downstream tasks (e.g., text generation, QA)",
    "⚖️ Trade-off Analysis: Find optimal balance between efficiency and quality",
    "🔄 Dynamic Masking: Explore adaptive masking based on input complexity"
]

for i, rec in enumerate(recommendations, 1):
    print(f"  {i}. {rec}")

# Save comprehensive results
evaluation_summary = {
    'schedule_efficiency': schedule_efficiency,
    'evaluation_results': evaluation_results,
    'comparison_data': comparison_df.to_dict() if not comparison_df.empty else {},
    'evaluation_date': time.strftime('%Y-%m-%d %H:%M:%S'),
    'recommendations': recommendations
}

with open('../results/mask_scheduled/comprehensive_evaluation.json', 'w') as f:
    # Handle numpy types for JSON serialization
    class NumpyEncoder(json.JSONEncoder):
        def default(self, obj):
            if isinstance(obj, np.integer):
                return int(obj)
            elif isinstance(obj, np.floating):
                return float(obj)
            elif isinstance(obj, np.ndarray):
                return obj.tolist()
            return super().default(obj)
    
    json.dump(evaluation_summary, f, indent=2, cls=NumpyEncoder)

print("\n✓ Comprehensive evaluation results saved to ../results/mask_scheduled/comprehensive_evaluation.json")
print("\n🎯 Mask schedule evaluation completed!")
print("\nNext steps:")
print("- Train models with different schedules on full datasets")
print("- Run comprehensive evaluation with larger test sets")
print("- Generate paper figures and final analysis")
print("- Write up results and conclusions")